## Converting HydroShoot Impulse Experiment data

In [1]:
import numpy as np
import pandas as pd

sys.path.insert(1, os.path.join(sys.path[0], '../../'))  # for importing local packages from src

In [2]:
"""Configure data sources"""

# EXPERIMENT_ROOT = '/thesis-cyborg-plants/results/fixed_predawn_7days_jun_jul_aug'
# EXPERIMENT_ROOT = '/media/mcndt/thesis/thesis-cyborg-plants/results/fixed_predawn_7days_jun_jul_aug'
EXPERIMENT_ROOT = 'F:/thesis-cyborg-plants/results/meteo_impulse_hydroshoot'

In [3]:
"""Loading dataset"""
from src.model.hydroshoot.hydroshoot_experiment import load_runs, HydroShootExperiment

assert(os.path.isdir(EXPERIMENT_ROOT))
runs = load_runs(EXPERIMENT_ROOT)
print(len(runs))
print(runs[0])
print('Targets:', runs[0].get_targets())
print('Variables:', runs[0].get_state_variables())

33
HydroShootExperiment(n_steps=240, state_size=360)
Targets: ('input_Tac', 'input_u', 'input_hs', 'input_Rg', 'output_Rg', 'output_An', 'output_E', 'output_Tleaf')
Variables: ('E', 'psi_head', 'An', 'Eabs', 'FluxC', 'Ci', 'u', 'Tlc', 'Flux', 'gs', 'gb', 'Ei', 'par_photo.dHd')


In [4]:
from src.util import get_dirs_in_directory

run_dirs = get_dirs_in_directory(EXPERIMENT_ROOT)

experiment_names = [d[15:-2] for d in run_dirs]

experiment_names[:11]

['2012-06-08__width0__0',
 '2012-06-08__width1__0',
 '2012-06-08__width1__1500',
 '2012-06-08__width2__0',
 '2012-06-08__width2__1500',
 '2012-06-08__width3__0',
 '2012-06-08__width3__1500',
 '2012-06-08__width4__0',
 '2012-06-08__width4__1500',
 '2012-06-08__width5__0',
 '2012-06-08__width5__1500']

### Converting dataset


In [5]:
"""Methods for converting data to DataFrame format (from previous notebook)"""

def HydroShoot_input_df(experiment: HydroShootExperiment):
  input_cols = experiment.inputs.loc[:, experiment.inputs.columns != 'time'].columns
  new_names = {col : f'input_{col}' for col in input_cols}
  input_df = experiment.inputs.rename(columns=new_names)
  input_df = input_df.reset_index(drop=True)
  input_df.insert(1, 'type', 'INPUT')
  return input_df


def HydroShoot_output_df(experiment: HydroShootExperiment):
  output_cols = experiment.outputs.loc[:, experiment.outputs.columns != 'time'].columns
  new_names = {col : f'output_{col}' for col in output_cols}
  output_df = experiment.outputs.rename(columns=new_names)
  output_df = output_df.reset_index(drop=True)
  output_df.insert(1, 'type', 'OUTPUT')
  return output_df


def HydroShoot_state_df(experiment: HydroShootExperiment):
  varnames = experiment.states.get_variables()
  state_ids = list(experiment.states._states[varnames[0]].keys())
  steps = experiment.n_steps()

  rows = []
  for step in range(steps):
    for state_id in state_ids:
      row = {'time': experiment.inputs['time'].iloc[step], 'type': 'STATE', 'state_id': state_id, 'state_type': 'leaf'}
      for varname in varnames:
        value = experiment.states._states[varname][state_id][step]
        row[f'state_{varname}'] = value
      rows.append(row)

  state_df = pd.DataFrame(rows)
  return state_df

In [6]:
"""Method for creating a single DataFrame for an experiment, including run id"""

def HydroShoot_experiment_df(experiment: HydroShootExperiment, run_id) -> pd.DataFrame:
  input_df = HydroShoot_input_df(experiment)
  output_df = HydroShoot_output_df(experiment)
  state_df = HydroShoot_state_df(experiment)
  experiment_df = pd.concat([input_df, output_df, state_df])
  experiment_df.insert(0, 'run_id', run_id)
  return experiment_df

dataset_df = HydroShoot_experiment_df(runs[0], experiment_names[0])
dataset_df.head(1)

,run_id,time,type,input_Tac,input_u,input_hs,input_Rg,output_Rg,output_An,output_E,...,state_Eabs,state_FluxC,state_Ci,state_u,state_Tlc,state_Flux,state_gs,state_gb,state_Ei,state_par_photo.dHd
0,2012-06-08__width0__0,2012-06-08,INPUT,17.58,0.418,82.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
"""Create a single dataframe for all experiment data across all runs."""
from typing import List

def HydroShoot_multirun_df(experiments: List[HydroShootExperiment], experiment_ids) -> pd.DataFrame:
  run_dfs = [HydroShoot_experiment_df(run, run_id=experiment_ids[i]) for i, run in enumerate(experiments)]
  return pd.concat(run_dfs, ignore_index=True)

dataset_df = HydroShoot_multirun_df(runs, experiment_names)

In [8]:
dataset_df.tail()


,run_id,time,type,input_Tac,input_u,input_hs,input_Rg,output_Rg,output_An,output_E,...,state_Eabs,state_FluxC,state_Ci,state_u,state_Tlc,state_Flux,state_gs,state_gb,state_Ei,state_par_photo.dHd
2867035,2012-08-19__width5__1500,2012-08-28 23:00:00,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,-0.009458,-2.131328e+07,0.755,20.076093,0.000000e+00,-2.778902e-14,0.770880,0.0,200.0
2867036,2012-08-19__width5__1500,2012-08-28 23:00:00,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,-0.008582,8.471491e+06,0.755,20.360084,1.142338e-19,6.990325e-14,0.779614,0.0,200.0
2867037,2012-08-19__width5__1500,2012-08-28 23:00:00,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,-0.007880,-5.103892e+06,0.755,20.159643,0.000000e+00,-1.162114e-13,0.789021,0.0,200.0
2867038,2012-08-19__width5__1500,2012-08-28 23:00:00,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,-0.007327,-8.839352e+06,0.755,20.158075,0.000000e+00,-6.706823e-14,0.798394,0.0,200.0
2867039,2012-08-19__width5__1500,2012-08-28 23:00:00,STATE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,-0.006509,-4.913865e+06,0.755,20.272962,0.000000e+00,-1.208258e-13,0.808110,0.0,200.0


In [9]:
dataset_df.to_csv('../datasets/hydroshoot_impulse.csv', index=False)